# Dueling Network Architecture Implementation
The Duelling newtork is an artificial neural network architecture that has improved the state of the art in the DQN area used in combination with Dual DQN and Prioritized Expirience Replay. This approach splits the action value calculation using a combination of state value function and advantage function.

# Searching for available environments
We want to test the performance of our architecture with the Atari game 'BeamRider', we can check wich kind of version of this game are available.

In [1]:
from gym import envs

# Searching for available environments
game_name = "Phoenix"
all_envs = envs.registry.values()
env_ids = [env_spec.id for env_spec in all_envs]

for id in sorted(env_ids):
    if game_name in id:
        print(id)

ALE/Phoenix-ram-v5
ALE/Phoenix-v5
Phoenix-ram-v0
Phoenix-ram-v4
Phoenix-ramDeterministic-v0
Phoenix-ramDeterministic-v4
Phoenix-ramNoFrameskip-v0
Phoenix-ramNoFrameskip-v4
Phoenix-v0
Phoenix-v4
PhoenixDeterministic-v0
PhoenixDeterministic-v4
PhoenixNoFrameskip-v0
PhoenixNoFrameskip-v4


/home/ste/anaconda3/lib/python3.9/site-packages/requests/packages/urllib3/_collections.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping, MutableMapping


# Environment Configuration
We select the version 4 of the enviroment with no frameskipping and select as render mode rgb array. The no frameskipping is used to make this enviroment compatible with the optimization made by *AtariPreprocessing*.

In [2]:
# from gym import envs
from gym.wrappers import AtariPreprocessing

# Make Parameters:
game_name = "Phoenix"
game_mode = "NoFrameskip"  # [Deterministic | NoFrameskip | ram | ramDeterministic | ramNoFrameskip ]
game_version = "v4"  # [v0 | v4 | v5]
env_name = '{}{}-{}'.format(game_name, game_mode, game_version)
env_render_mode = 'rgb_array'  # [human | rgb_array]
env_frame_skip = 4

env = envs.make(env_name, render_mode=env_render_mode)

A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]
/home/ste/anaconda3/lib/python3.9/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/ste/anaconda3/lib/python3.9/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
env.reset() 
env.render(mode="rgb_array")

/home/ste/anaconda3/lib/python3.9/site-packages/gym/core.py:51: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/home/ste/anaconda3/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:297: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[146,  70, 192],
        [146,  70, 192],
        [146,  70, 192],
        ...,
        [146,  70, 192],
        [146,  70, 192],
        [146,  70, 192]],

       [[146,  70, 192],
        [146,  70, 192],
        [146,  70, 192],
        ...,
        [146,  70, 192],
        [146,  70, 192],
        [146,  70, 192]],

       [[146,  70, 192],
        [146,  70, 192],
        [146,  70, 192],
        ...,
        [146,  70, 192],
        [146,  70, 192],
        [146,  70, 192]]

## Envorment Observations
Below we have a fist look to the enviroment characteristics.

In [4]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

In [5]:
env.action_space

Discrete(8)

In [6]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'DOWN', 'RIGHTFIRE', 'LEFTFIRE', 'DOWNFIRE']

## Enviroment Optimization
We optimize the enviroment adding the frame skipping, changing its observation in greyscale and following the experiment did in the paper we set to at most 30 the no-op actions; to get this we use the AtariPreprocessing wrapper.

In [7]:
env_prep = AtariPreprocessing(env, frame_skip=env_frame_skip, grayscale_obs=True, noop_max=30)

In [8]:
env_prep.reset()

/home/ste/anaconda3/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [107, 107, 107, ..., 107, 107, 107],
       [107, 107, 107, ..., 107, 107, 107],
       [107, 107, 107, ..., 107, 107, 107]], dtype=uint8)

In [9]:
env_prep.observation_space.shape

(84, 84)

In [10]:
env_prep.action_space

Discrete(8)

# Network configuration

## Network parameters

In [11]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import losses 
from tensorflow.keras import optimizers

input_shape = env_prep.observation_space.shape
num_actions = env_prep.action_space.n

2022-09-30 19:40:59.420780: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/home/ste/anaconda3/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:513: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
/home/ste/anaconda3/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:521: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.htm

## Policy
The policy is the component that choose the action to perform; using an $\epsilon$-gready policy the action choosen can be a radom one with probability $\epsilon$ and an action suggested by the ANN with probability $1 - \epsilon$.
From paper: " When acting, it suffices to evaluate the advantage stream to make decisions."

In [12]:
import numpy as np


class Policy:
    def __init__(self, model):
        self.model = model
        pass

    def get_action(self, state):
        pass


class EpsilonGreedyPolicy(Policy):

    def __init__(self, model, action_space_size, episodes=1, min_epsilon=0):
        super().__init__(model)
        self.action_space_size = action_space_size
        self.min_epsilon = min_epsilon
        self.episode = 1
        self.episodes = episodes

    def get_action(self, state):
        epsilon = max(1 - self.episode / self.episodes, self.min_epsilon)
        random = np.random.random()
        # print(random, epsilon)
        if random < epsilon:
            action = np.random.randint(self.action_space_size)
            return action
        else:
            state = state[np.newaxis]
            q_values = self.model(state)
            # q_values = self.model(state[np.newaxis])
            action = np.argmax(q_values[0])
            return action

    def next_episode(self):
        self.episode += 1

    def reset_episodes(self):
        self.episode = 1


## Neural Network Creation
The creation of the DQN Model is based on the example presented in https://keras.io/examples/rl/deep_q_network_breakout/. The network architecture proposed follows the structure proposed in *"Dueling Network Architectures for Deep Reinforcement Learning"* https://arxiv.org/abs/1511.06581 composed by 3 convolutional layers and 2 fully connected layer for each stream (advantage, value).
It's possible to create a dueling network using the `DQNAgent` of `rl.agents.dqn` setting `enable_dueling_network=True` in the constructor, but the perpouse of this experiment is to show how to develop it manually so that is not used.

The output of the value stream and the output of the advantage stream are merged to obtain the action-value function in the last module of the network using the following formula:
$$ Q(s, a; \theta, \alpha, \beta) = V (s;\theta, \beta) + ( A(s, a; \theta, \alpha) − max_{a' ∈|A|} A(s, a'; \theta, \alpha)) $$

Following the paper: 
"Since both the advantage and the value stream propagate gradients to the last convolutional layer in the backward pass, we rescale the combined gradient entering the last convolutional layer by $\frac{1} {\sqrt2}$."

In [21]:
from tensorflow import math
from tensorflow.keras import layers
from tensorflow.keras import Model


def create_dueling_model(input_shape, number_actions):
    inputs = layers.Input(shape=input_shape)

    # Convolutions on the frames on the screen
    layer1 = layers.Conv1D(32, 8, strides=4, activation="relu")(inputs)
    layer2 = layers.Conv1D(64, 4, strides=2, activation="relu")(layer1)
    layer3 = layers.Conv1D(64, 3, strides=1, activation="relu")(layer2)
    layer4 = layers.Flatten()(layer3)

    value_stream_1 = layers.Dense(512)(layer4)
    value_stream_2 = layers.Dense(1)(value_stream_1)  # scalar output size

    advantage_stream_1 = layers.Dense(512)(layer4)
    advantage_stream_2 = layers.Dense(number_actions)(advantage_stream_1)  # output size equal to the actions available

    # Combination of the streams: a Q value for each state
    q_values = value_stream_2 + math.subtract(advantage_stream_2, math.reduce_mean(advantage_stream_2, axis=1,
                                                                                   keepdims=True))
    # Alternative q_value
    # q_value = value_stream_2 + (advantage_stream_2 - backend.max(advantage_stream_2, axis=1, keepdims=True))
    return Model(inputs=[inputs], outputs=[q_values])


## Replication Buffer

In [22]:
class ReplayBuffer:
    def __init__(self, max_buffer_size):
        self.max_buffer_size = max_buffer_size
        pass

    def sample_experience(self, batch_size):
        pass

### Prioritized experience replay
The Prioritized experience replay was introduced in the paper "Prioritized experience replay" (https://arxiv.org/abs/1511.05952), it consists in an evolution in the replay buffer usage, ordering by priority the experience to replay. In this experiment we adopt the **rank-based** variant where the experience sampling from the buffer it's done with probability $ P(i) = \frac{p_{i}^{\alpha}}{\sum_k{p_{k}^{\alpha}}} $ and $p(i)=\frac{1}{rank(i)}$ where $rank(i)$ is the rank of the -ith transition when the buffer is sorted by the TD error $\delta$ of each experience; $\alpha$ is called **priority exponent**. Is also compute the importance sampling weights as $w_j = \frac{(N . P(i))^{-\beta}}{max_i{w_i}} $ and $w_i = (\frac{1}{N} . \frac{1}{P(i)})^\beta$

In both the paper the parameters are setted as follow: **priority exponent** $\alpha= 0.7$,  the **importance sampling exponent** $\beta = [0.5, 1]$.
In the paper is proposed a **heap array** structure to implement the buffer. Due to the particular structure and the amount of property of the replay buffer in the Prioritized Experience Replay, we choose to describe it as a class. The heap array structure is implemented as a deque sorted every *T* step.

In the Prioritized experience replay paper another kind of implementation in propoused that consistes in the divition of the buffer in a *batchsize* parts and sampling from them; this garantees to have a sample for each rank category. We follow the first implementation using the numpy *zipf* function.

In [23]:
# import heapq as heap
import numpy as np
import scipy.stats as stats

# We set a time to haepify to sort the buffer every K time step.
class PrioritizedExperienceReplayRankBased(ReplayBuffer):
    """
    contains the tuples (TD_error, experience)
    replay_buffer --- it's the max size of the buffer, over which before add an experience one is remove
    max_buffer_size --- time step before sort the structure
    time_to_haepify --- the last time step
    mod_curr_step = 0  --- the alpha parameter used to calculate the probability of the i-th element P(i) to be sampled
    alpha -- alpha parameter
    """

    def __init__(self, max_buffer_size, step_to_heapify, alpha):
        super().__init__(max_buffer_size=max_buffer_size)
        # (TD, experience)
        # Probably list is not the most efficient structure to use np array ?
        self.replay_buffer = []
        self.alpha = alpha
        self.heapify_threshold = step_to_heapify  # here we stock the threshold to sort the buffer
        self.step_to_heapify = step_to_heapify  # number of next steps before heapify
        self.max_td_error = 0

    def set_alpha(self, alpha):
        self.alpha = alpha

    # Add experience in the buffer mapping it with its last TD_error
    def add_experience(self, experience):
        if len(self.replay_buffer) == self.max_buffer_size:
            self.remove_experience()

        # New experience where td_error is unknown are set with the max td error
        # NB we are considering the max td error as the error of the experience in first position, but the buffer may
        # not have been sorted yet
        if len(self.replay_buffer) > 0:
            self.max_td_error = self.replay_buffer[0][0]

        self.replay_buffer.append((self.max_td_error, experience))
        self.step_to_heapify -= 1
        if self.step_to_heapify == 0:
            self.replay_buffer.sort(key=lambda el: el[0], reverse=True)
            self.step_to_heapify = self.heapify_threshold

    # Remove experience from the buffer
    def remove_experience(self):
        self.replay_buffer.pop(-1)

    @staticmethod
    def zip_f_sampling(alpha, n):
        x = np.arange(1, n + 1)
        weights = x ** (-alpha)
        weights /= weights.sum()
        zipf = stats.rv_discrete(values=(x, weights))
        return zipf.rvs() - 1

    # Get batch_size samples from the buffer; using the beta parameter to compute the importance sampling weight
    # Beta value can change while training we can delegate its control outside
    def sample_experience(self, batch_size, beta):
        experiences = []
        importance_sampling_weights = []
        n = len(self.replay_buffer) - 1
        indexes = []

        for i in range(0, batch_size):
            # Sample index and check the experience is not already present in the batch
            index = self.zip_f_sampling(self.alpha, n)
            while index in indexes:
                index = self.zip_f_sampling(self.alpha, n)
            indexes.append(index)
            # importance sampling weights computation
            rank = index + 1
            pj = 1 / rank
            importance_sampling_weights.append(((n * pj) ** (-beta)))
            experiences.append(self.replay_buffer[index][1])

        # Normalization step
        max_weight = max(importance_sampling_weights)
        importance_sampling_weights_normalized = np.divide(importance_sampling_weights, max_weight)
        return indexes, experiences, importance_sampling_weights_normalized

    def update_td_error(self, index, td_error):
        self.replay_buffer[index] = [td_error, self.replay_buffer[index][1]]


# Agent 
Here we define a custom agent to perfome action in the enviroment using a DoubleDQN training

## Play one step
With this function we want to ask to the policy what action must be choosen and perform it on the evniroment.

## Gradient 
The gradient is the mathematical function that calcualte the weights update in the ANN following the gradient minimization tecnique. In our scenario the gradient that is backpropageted to the last convolutional layer must be rescaled by $\frac{1}{\sqrt{2}}$

## Double DQN Training
This training algorithm uses a second network, beyond the network used for the prediction. So in the training process the main network is used to choose an action and an other to evaluate it, this permit to mitigate the overfitting present in the classic DQN. 

## Play

In [24]:
import math as mt

import tensorflow as tf
import numpy as np


class DuelDQNAgent:

    ## We keep the creation model outside the agent to ensure a fine-grained control on it
    def __init__(self, env, model, policy, model_target=None, optimizer=None, replay_buffer=None):
        self.env = env
        self.model_primary = model
        self.model_target = model_target
        self.policy = policy
        self.optimizer = optimizer
        self.replay_buffer = replay_buffer

    def set_policy(self, policy):
        self.policy = policy

    # Execs one action receiving in input the environment, its state, the current episode.
    # If training its true add the experience in the replay buffer
    def play_one_step(self, state):
        action = self.policy.get_action(state)
        # print("action {}".format(action))
        next_state, reward, done, info = self.env.step(action)
        return action, reward, next_state, done, info
    
    def weighted_gradient(self, best_on_target_q_values, importance_sampling_weights, states, loss_function, mask,
                          step_size=1):
        with tf.GradientTape() as tape:
            tape.watch(importance_sampling_weights)
            all_q_values = self.model_primary(states)
            q_values = tf.reduce_sum(all_q_values * mask, axis=1, keepdims=True)
            loss_value = loss_function(best_on_target_q_values, q_values)
            loss_corrected = tf.multiply(loss_value, importance_sampling_weights, step_size)
        grads = tape.gradient(loss_corrected, self.model_primary.trainable_variables)
        return grads, loss_value

    @staticmethod
    def rescale_grad(gradients, rescale_value, index):
        tensor_to_scale = gradients[index]
        rescaled_tensor = tf.multiply(tensor_to_scale, rescale_value)
        gradients[index] = rescaled_tensor
        return gradients

    @staticmethod
    def gradient_clipping(gradients, clipping_value):
        clipped_gradients = [(tf.clip_by_norm(grad, clipping_value)) for grad in gradients]
        return clipped_gradients
    
    # Collects samples of the previous experiences from the replay buffer
    # and use them to improve the weights update of the Neural Network.
    def double_dqn_training_step(self, batch_size, loss_function, discount_factor, clipping_value, beta, step_size=1):
        indexes, experiences, importance_sampling_weights = self.replay_buffer.sample_experience(batch_size, beta)
        states, actions, rewards, next_states, dones = [np.array([experience[field_index] for experience in experiences]
                                                                 ) for field_index in range(5)]

        action_space = self.env.action_space.n
        # Predict using the primary network
        next_q_values = self.model_primary.predict(next_states)
        next_q_values_target = self.model_target.predict(next_states)

        # Select the action that lead us to the higher next Q value
        best_actions = np.argmax(next_q_values, axis=1)
        best_action_mask = tf.one_hot(best_actions, action_space)

        next_q_value_target = tf.reduce_sum(next_q_values_target * best_action_mask, axis=1)
        best_on_target_q_values = (rewards + (1-dones)*discount_factor*next_q_value_target)

        mask = tf.one_hot(actions, action_space)
        importance_sampling_weights = tf.convert_to_tensor(importance_sampling_weights, tf.float32)
        weighted_gradient, loss_value = self.weighted_gradient(best_on_target_q_values, importance_sampling_weights,
                                                               states, loss_function, mask, step_size)

        for index, td_error in zip(indexes, loss_value):
            self.replay_buffer.update_td_error(index, td_error)

        # We rescale the last convolutional layer to 1/sqrt(2) to balance the double backpropagation
        rescale_value = (1 / mt.sqrt(2))
        # The index of the last sequential layer
        index_gradient_to_rescale = 4
        rescaled_grads = self.rescale_grad(weighted_gradient, rescale_value, index_gradient_to_rescale)

        # Since we are in a custom loop we have to clip the gradient by hand, we can't delegate it to the optimizer
        clipped_gradients = self.gradient_clipping(rescaled_grads, clipping_value)
        # Application gradient descent trough optimizer
        self.optimizer.apply_gradients(zip(clipped_gradients, self.model_primary.trainable_variables))

    # We use the training step just when there is enough samples on the replay buffer
    def double_dqn_training(self, batch_size, loss_function, discount_factor, freq_replacement, clipping_value,
                            beta_min, beta_max, max_episodes=600):
        rewards = []
        steps = []

        for episode in range(1, max_episodes+1):
            state = self.env.reset()
            cumulative_reward = 0
            step = 0
            beta = max(beta_min, (beta_max * episode / max_episodes))
            while True:
                action, reward, next_state, done, info = self.play_one_step(state)
                experience = [state, action, reward, next_state, done]
                cumulative_reward += reward
                self.replay_buffer.add_experience(experience)
                if done:
                    print(
                        "DONE episode = {} number of steps = {} reward = {}".format(episode, step, cumulative_reward))
                    rewards.append(cumulative_reward)
                    steps.append(step)
                    break
                if len(self.replay_buffer.replay_buffer) > batch_size:
                    self.double_dqn_training_step(batch_size, loss_function, discount_factor, clipping_value, beta)
                if step == freq_replacement:
                    self.model_target.set_weights(self.model_primary.get_weights())
                state = next_state
                step = step + 1

            self.policy.next_episode()

        return steps, rewards

        def play(env, model, min_epsilon, action_space_size):   
            while True:
                env_prep.render(mode=env_render_mode)
                epsilon = 0
                obs, reward, done, info = play_one_step(env, obs, None, min_epsilon, training = False)

                if done:
                    print("DONE episode = {} number of steps = {} reward =  {}".format( episode, step, cumulative_reward))
                    rewards.append(reward)
                    break

            print('last episode = {} reward = {}'.format(episode, reward))
            return episode, reward


# Training
The learnig step is executed with **Double Deep Q-networks** algorithm presented in the paper *"Deep reinforcement learning with double Q-learning"*.https://arxiv.org/pdf/1509.06461.pdf.

## Training parameters
We adopt as optimizer the **Adam** implementation setting the learning rate equal to $6.25x10^{-5}$ and **clipping the gradient** norm at most to 10; the parameters are specified in the paper "*Deep reinforcement learning with double Q-learning*" (https://arxiv.org/pdf/1509.06461.pdf)
To evaluate the loss score we use the `mean_squared_error`. 

In [25]:
# Network Parameters
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras.models import load_model

# Environment info
input_shape = env_prep.observation_space.shape
actions_number = env_prep.action_space.n

# Model persistent file
primary_model_file_name = "{}_dueling_model".format(game_name)

# Training Parameters
loss_function = losses.mean_squared_error
batch_size = 32
discount_factor = 0.95
learning_rate = 6.25e-5
episodes = 50
clipping_value = 10

# Dual DQN Training
freq_replacement = 50

# Replay buffer parameters
buffer_size = 2000
step_to_heapify = 50
alpha = 0.5
beta_max = 1
beta_min = 0.5

# Policy parameters
min_epsilon = 0.001


## Result plots
We use this function to generate the plot representing the rewards or the steps for each episode

In [26]:
def plot_result(max_episodes,returns):
    plt.ylabel('Reward')
    plt.xlabel('Episode')
    plt.plot(max_episodes, returns)
    plt.show()
    return

## Model creation or loading 
In this step we check wheter there is an already saved model and loading it in that case

In [27]:
from pathlib import Path
import pandas as pd

# Model creation
file_primary = Path(primary_model_file_name)
if file_primary.exists():
    print("Found an existing model")
    model = load_model(primary_model_file_name)
else:
    print("Model not found, a new one will be crate")
    model = create_dueling_model(input_shape, actions_number)

# Print a summary about the model
print(model.summary())

Model not found, a new one will be crate
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 84, 84)]     0                                            
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 20, 32)       21536       input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 9, 64)        8256        conv1d_6[0][0]                   
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 7, 64)        12352       conv1d_7[0][0]                   
___________________________________________________

## Run Training
Here we ran the training operation. After a training session we save two plot episode - rewards, episode - steps. Also we save a csv with two column: steps and rewards.

In [28]:
# Setting the optimizer
from tensorflow import math

training = True
if training:
    model_target = create_dueling_model(input_shape, actions_number)
    model_target.set_weights(model.get_weights())
    optimizer = optimizers.Adam(learning_rate=learning_rate)
    policy_training = EpsilonGreedyPolicy(model, actions_number, episodes=episodes, min_epsilon=min_epsilon)
    replay_buffer = PrioritizedExperienceReplayRankBased(buffer_size, step_to_heapify, alpha)
    agent = DuelDQNAgent(env_prep, model, policy_training, model_target, optimizer, replay_buffer)
    steps, rewards = agent.double_dqn_training(batch_size, loss_function, discount_factor, freq_replacement,
                                               clipping_value, beta_min, beta_max, episodes)
    env_prep.close()
    model.save(primary_model_file_name)

    ext = "png"
    name_plot_eps_steps = "{} Training Episodes Steps.{}".format(game_name, ext)
    name_plot_eps_rewards = "{} Training Episodes Rewards.{}".format(game_name, ext)
    file_plot_1 = Path(name_plot_eps_steps)
    file_plot_2 = Path(name_plot_eps_rewards)
    i = 1
    while file_plot_1.exists():
        file_plot_1 = Path(name_plot_eps_steps)
        name_plot_eps_steps = "{} Training Episodes Steps_{}.{}".format(game_name, i, ext)
        name_plot_eps_rewards = "{} Training Episodes Rewards_{}.{}".format(game_name, i, ext)
        i += 1

    plot_result("Episode", "Steps", range(1, episodes+1), steps, name_plot_eps_steps)
    plot_result("Episode", "Steps", range(1, episodes+1), rewards, name_plot_eps_rewards)


    csv_name = "{}.csv".format(game_name)
    dict = {'steps': steps, 'rewards': rewards}
    df = pd.DataFrame(dict)
    df.to_csv(csv_name, mode='a', header=False)


DONE episode = 1 number of steps = 896 reward = 320.0


KeyboardInterrupt: 

# Run play
Here we play a game (one episode), save two plot episode - rewards, episode - steps. Also we save a csv with two column: steps and rewards.

In [ ]:
# Play
play = False
if play:
    policy_play = EpsilonGreedyPolicy(model, actions_number, min_epsilon=min_epsilon)
    agent = DuelDQNAgent(env_prep, model, policy_play)
    steps, reward = agent.play()

    ext = "png"
    name_plot_steps_rewards = "{} Play Steps Rewards.{}".format(game_name, ext)
    file_plot_1 = Path(name_plot_steps_rewards)
    i = 1
    while file_plot_1.exists():
        file_plot_1 = Path(name_plot_steps_rewards)
        name_plot_steps_rewards = "{} Play Steps Rewards_{}.{}".format(game_name, i, ext)
        i += 1
    plot_result("Episode", "Steps", steps, reward, name_plot_steps_rewards)
    